In [7]:
import streamlit as st
import pandas as pd
import mysql.connector
import re

def parse_voting_string(value):
    try:
        if pd.isna(value):
            return 0
        value = str(value).strip().replace(",", "").replace("(", "").replace(")", "").upper()
        match = re.match(r'^([\d\.]+)([KMB]?)$', value)
        if not match:
            return 0
        num, suffix = match.groups()
        num = float(num)
        if suffix == 'K':
            return int(num * 1_000)
        elif suffix == 'M':
            return int(num * 1_000_000)
        elif suffix == 'B':
            return int(num * 1_000_000_000)
        else:
            return int(num)
    except:
        return 0
def parse_duration(value):
    try:
        if pd.isna(value):
            return 0
        value = str(value).lower().replace("hrs", "").replace("hr", "").replace("hours", "").strip()
        return float(re.findall(r"[\d.]+", value)[0])
    except:
        return 0
def load_movies_data():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="imdb_genre_wise"
    )
    query = "SELECT title, Genre, rating, voting, Duration FROM movies_2024"
    df = pd.read_sql(query, conn)
    conn.close()   
    return df

df= load_movies_data()
df.columns = [col.strip().capitalize() for col in df.columns]
df["Voting"] = df["Voting"].apply(parse_voting_string)
df["Duration"] = df["Duration"].apply(parse_duration)
if df.empty or df["Voting"].max() <= 0:
    st.error("No valid voting data found. Please check your database or parsing logic.")
    st.dataframe(df)
    st.stop()
st.sidebar.header(" Filter Movies")

unique_genres = sorted(df["Genre"].dropna().unique())
selected_genre = st.sidebar.selectbox(" Select Genre:", ["All"] + list(unique_genres))

# rating Filter
min_rating = float(df["Rating"].min())
max_rating = float(df["Rating"].max())
selected_rating = st.sidebar.slider(" Minimum IMDb Rating:", min_value=min_rating, max_value=max_rating, value=min_rating)

# voting Filter
min_voting = int(df["Voting"].min())
max_voting = int(df["Voting"].max())
selected_voting = st.sidebar.slider(" Minimum voting:", min_value=min_voting, max_value=max_voting, value=min_voting)

# Duration Filter
duration_filter = st.sidebar.radio(" Movie Duration:", ["< 2 hrs", "2–3 hrs", "> 3 hrs"])
filtered_df = df.copy()

if selected_genre != "All":
    filtered_df = filtered_df[filtered_df["Genre"] == selected_genre]

filtered_df = filtered_df[filtered_df["Rating"] >= selected_rating]
filtered_df = filtered_df[filtered_df["Voting"] >= selected_voting]

if duration_filter == "< 2 hrs":
    filtered_df = filtered_df[filtered_df["Duration"] < 2]
elif duration_filter == "2–3 hrs":
    filtered_df = filtered_df[(filtered_df["Duration"] >= 2) & (filtered_df["Duration"] <= 3)]
else:
    filtered_df = filtered_df[filtered_df["Duration"] > 3]


# Display Results
st.success(f"🎉 Found {len(filtered_df)} matching movies!")
st.dataframe(filtered_df)

C:\Users\Raj Kumar R\AppData\Local\Temp\ipykernel_20996\2806957968.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
2025-09-06 02:59:27.734 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-06 02:59:27.736 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-06 02:59:27.739 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-06 02:59:27.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-06 02:59:27.744 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-06 02:59:27.745 Thread 'MainThread': missing ScriptRunC

DeltaGenerator()